# **Install dan Setting Library**

- Setting selenium khusus Google Colab: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
- Tidak perlu run kode cell ini jika melakukan running di Jupyter Notebook atau local Python lain

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,590 kB]
Get:14 http://archive.ubuntu.com/ubunt

In [ ]:
!pip3 install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 81.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.1 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=bee76686adc35f6d37dcd187661ca710fc0ae474ad6fcbff250a91382126f8f9
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3342 sha256=4cfdb759a536e85edb36ced110fd4833f5f4b3fdaca962d4a7887f68a4bb8a16
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc933e4

In [ ]:
!pip3 install chromedriver-autoinstaller

In [ ]:
# Import library
import re
import csv
import sys
import time
import pandas as pd
import chromedriver_autoinstaller
from newspaper import Article
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Setting chrome driver
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/129/chromedriver'

# **Load CSV Link Berita**

In [ ]:
from google.colab import files
upload = files.upload()

Saving link_indozone.csv to link_indozone.csv


In [ ]:
# Import daftar link berita menjadi DataFrame
df_news_link = pd.read_csv('link_indozone.csv')
df_news_link

,Link,Tanggal
0,https://news.indozone.id/news/915064988/lansia...,07-09-2024
1,https://news.indozone.id/indonesia-hari-ini/91...,21-06-2024
2,https://news.indozone.id/news/914774986/alasan...,19-06-2024
3,https://news.indozone.id/news/914726440/iuran-...,05-06-2024
4,https://news.indozone.id/news/914724357/3-cara...,04-06-2024
...,...,...
165,https://news.indozone.id/news/911466282/menkeu...,28-05-2019
166,https://news.indozone.id/news/911465764/biaya-...,21-05-2019
167,https://news.indozone.id/news/911465352/rumah-...,15-05-2019
168,https://news.indozone.id/news/911464506/may-da...,01-05-2019


# **Scraping dan Parsing Teks Berita**

Menguraikan judul, URL link, konten, tanggal, tags dari laman artikel berita.

In [ ]:
from bs4 import BeautifulSoup
from newspaper import Article
from selenium import webdriver
import time

def scrape_news_text(df_news_link):

    dict_news = dict()
    status = list()

    for i in df_news_link.index:

        # Akses dan download article
        article = Article(str(df_news_link['Link'][i]), language='id')
        article.download()

        # Jika download berhasil, uraikan HTML
        if article.download_state == 2:
            article.parse()

            # Hapus elemen yang tidak diinginkan sebelum mengambil teks
            soup = BeautifulSoup(article.html, 'html.parser')

            # Hapus elemen-elemen tertentu berdasarkan class atau tag
            for selector in ['div.photo__caption', 'strong.read__others', 'strong em']:
                for tag in soup.select(selector):
                    tag.decompose()  # Menghapus elemen dari DOM

            # Set HTML yang sudah bersih
            article.set_html(str(soup))
            article.parse()

            # Jika teks tidak kosong, ambil isinya
            if article.text:
                print(f"{i+1} [success] {article.url}")
                status.append('success')
                dict_news[i] = {
                    'title': str(article.title),
                    'url': str(article.url),
                    'text': ' '.join(article.text.splitlines()),  # Gabungkan teks agar tidak terpisah
                    'date': df_news_link['Tanggal'][i] if 'Tanggal' in df_news_link.columns else 'N/A',
                    'tags': ', '.join(article.tags)
                }
                continue

        # Jika gagal, maka ambil HTML-nya secara manual pakai Selenium
        input_html = ''
        driver = webdriver.Chrome(options=chrome_options)

        try:
            driver.get(str(df_news_link['url'][i]))
            input_html = driver.page_source
            time.sleep(2)

            # Gunakan BeautifulSoup untuk mengekstrak tanggal dan teks
            soup = BeautifulSoup(input_html, 'html.parser')

            # Hapus elemen-elemen yang tidak diinginkan
            for selector in ['div.photo__caption', 'strong.read__others', 'strong em']:
                for tag in soup.select(selector):
                    tag.decompose()  # Hapus elemen

            date_element = soup.find('div', class_='read__info__date')
            date_text = date_element.get_text(strip=True) if date_element else 'N/A'

            article.download(input_html)
            article.parse()

            # Jika teks tidak kosong, ambil isinya
            if article.text:
                print(f"{i+1} [success] {article.url}")
                status.append('success')
                dict_news[i] = {
                    'title': str(article.title),
                    'url': str(article.url),
                    'text': ' '.join(article.text.splitlines()),  # Gabungkan teks agar tidak terpisah
                    'date': date_text,
                    'tags': ', '.join(article.tags)
                }
                continue
        except Exception as e:
            pass
        finally:
            driver.close()

        # Jika HTML atau teks-nya kosong maka FAILED
        print(f"{i+1} [failed ] {article.url}")
        status.append('failed')

    return (dict_news, status)


In [ ]:
# Scraping dan parsing text berita
dict_news, status = scrape_news_text(df_news_link)

1 [success] https://news.indozone.id/news/915064988/lansia-di-jaksel-rugi-rp1-m-usai-jadi-korban-penipuan-pelaku-pura-pura-jadi-petugas-bpjs-dan-polisi
2 [success] https://news.indozone.id/indonesia-hari-ini/914783072/respon-abdi-dalem-keraton-jogja-soal-wacana-bpjs-ketenagakerjaan
3 [success] https://news.indozone.id/news/914774986/alasan-dprd-kota-jogja-usulkan-abdi-dalem-dapat-bpjs-ketenagakerjaan
4 [success] https://news.indozone.id/news/914726440/iuran-bpjs-kesehatan-bisa-dicairkan-jika-peserta-tak-pernah-sakit-ini-penjelasannya
5 [success] https://news.indozone.id/news/914724357/3-cara-cetak-kartu-bpjs-kesehatan-secara-online-dengan-mudah-bisa-via-hp
6 [success] https://news.indozone.id/indonesia-hari-ini/914723055/catat-mulai-juli-2024-bpjs-jadi-syarat-bikin-dan-perpanjang-sim
7 [success] https://news.indozone.id/news/914692551/lupa-nomor-bpjs-ketenagakerjaan-tenang-begini-4-cara-mudah-mengeceknya
8 [success] https://news.indozone.id/news/914658111/daftar-iuran-bpjs-kesehatan-ke

# **Simpan Teks Berita ke CSV**

In [ ]:
# Mengonversi dictionary ke dalam DataFrame
df_news = pd.DataFrame.from_dict(dict_news, orient='index')
df_status = pd.DataFrame(status, columns=['status'])

In [ ]:
df_news.head(10)

,title,url,text,date,tags
0,Lansia di Jaksel Rugi Rp1 M Usai Jadi Korban P...,https://news.indozone.id/news/915064988/lansia...,INDOZONE.ID - Seorang lansia berinisial HS (75...,07-09-2024,"Bank, bank, lansia, polisi, penipuan, pegawai ..."
1,Respon Abdi Dalem Keraton Jogja Soal Wacana BP...,https://news.indozone.id/indonesia-hari-ini/91...,INDOZONE.ID - Salah satu warga Yogyakarta yang...,21-06-2024,"bpjs ketenagakerjaan, yogyakarta, abdi dalem, ..."
2,Alasan DPRD Kota Jogja Usulkan Abdi Dalem Dapa...,https://news.indozone.id/news/914774986/alasan...,INDOZONE.ID - Dewan Perwakilan Rakyat Daerah (...,19-06-2024,"DPRD Kota Yogyakarta, Keraton, dprd kota yogya..."
3,Iuran BPJS Kesehatan Bisa Dicairkan Jika Peser...,https://news.indozone.id/news/914726440/iuran-...,INDOZONE.ID - BPJS Kesehatan adalah program as...,05-06-2024,"BPJS Kesehatan, iuran bpjs kesehatan, peserta ..."
4,3 Cara Cetak Kartu BPJS Kesehatan Secara Onlin...,https://news.indozone.id/news/914724357/3-cara...,INDOZONE.ID - proses pencetakan kartu BPJS Kes...,04-06-2024,"kartu BPJS Kesehatan, Kartu BPJS Kesehatan, Ce..."
5,Catat! Mulai Juli 2024 BPJS Jadi Syarat Bikin ...,https://news.indozone.id/indonesia-hari-ini/91...,INDOZONE.ID - Korps Lalu Lintas (Korlantas) Po...,04-06-2024,"BPJS Kesehatan Syarat Bikin SIM, sim, Syarat B..."
6,"Lupa Nomor BPJS Ketenagakerjaan? Tenang, Begin...",https://news.indozone.id/news/914692551/lupa-n...,INDOZONE.ID - BPJS Ketenagakerjaan adalah lemb...,27-05-2024,"bpjs ketenagakerjaan, nomor BPJS, BPJS, nomor ..."
7,Daftar Iuran BPJS Kesehatan Kelas Rawat Inap 1...,https://news.indozone.id/news/914658111/daftar...,INDOZONE.ID - Seiring dengan rencana penerapan...,16-05-2024,"iuran BPJS Kesehatan, kelas 1, KRIS, BPJS Kese..."
8,Presiden Jokowi Resmi Hapus Klasifikasi Kelas ...,https://news.indozone.id/news/914649464/presid...,INDOZONE.ID - Presiden Joko Widodo secara resm...,14-05-2024,"Presiden Jokowi Hapus Klasifikasi Kelas BPJS, ..."
9,"Cara Cetak Kartu BPJS, Bisa Lewat HP Tidak Per...",https://news.indozone.id/news/914646806/cara-c...,"INDOZONE.ID - Sebagai masyarakat di Indonesia,...",14-05-2024,"cara cetak kartu BPJS yang hilang, cara cetak ..."


In [ ]:
# Menyimpan DataFrame ke dalam file CSV
output_file = 'newstext_indozone.csv'
df_news.to_csv(output_file, index=False, sep=";")
print(df_status['status'].value_counts())

status
success    165
failed       5
Name: count, dtype: int64
